# Simple RL code to teach the model to stand

## Setup jax enviroment and Load Model 

In [1]:
import numpy as np
import mediapy as media
import matplotlib.pyplot as plt


import os
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".60"

# Optionally, force JAX to preallocate memory.
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

# Setup environment variable for Nvidia GPU acceleration
os.environ['XLA_FLAGS'] = (
    '--xla_gpu_triton_gemm_any=True '
)

backend = 'gpu'
# backend = 'METAL'
# backend = 'cpu'

import jax
# Enable compliation catch
jax.config.update("jax_compilation_cache_dir", "./jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

# Debug Nan
# jax.config.update("jax_debug_nans", True)

from jax import numpy as jp
# More legible printing from numpy.
jp.set_printoptions(precision=4, suppress=True, linewidth=100)

import mujoco
import mujoco.mjx as mjx
from mujoco.mjx._src import scan
from mujoco.mjx._src import types

# More legible printing from numpy.
np.set_printoptions(precision=4, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

device = jax.devices(backend=backend)[0]

model_path = '/home/bugman/Currentwork/biomujoco_converter/converted/mjc/Gait2354/gait2354_cvt1.xml'

# Single step
mjx_step = jax.jit(mjx.step, backend=backend)



# mjx_multiple_steps = jax.jit(multiple_steps, backend=backend, )

In [18]:
from mujoco.mjx._src.biomtu.acceleration_mtu import acceleration_mtu

mj_model = mujoco.MjModel.from_xml_path(model_path)
mjx_model = mjx.put_model(mj_model,device=device)

# Disable tendon
opt = mjx_model.opt.replace(disableflags = mjx_model.opt.disableflags |mujoco.mjtDisableBit.mjDSBL_PASSIVE)
mjx_model = mjx_model.replace(opt=opt)

mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
mj_data.qpos = mj_model.key_qpos[0]

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
mjx_data = mjx_step(mjx_model, mjx_data)

def print_all():
    print(mjx_model.biomtu_adr)
    print(mjx_model.mtu_wrap_objid)
    print(mjx_model.mtu_wrap_type)
    print(mjx_model.biomtu_fiso)
    print(mjx_model.biomtu_vmax)
    print(mjx_model.biomtu_ofl)
    print(mjx_model.biomtu_opa)
    print(mjx_model.biomtu_mass)
    print("-------Data--------")
    print("qpos:", mjx_data.qpos)
    print("mtu l:", mjx_data.biomtu.l)
    print("tendon l:", mjx_data.biomtu.tendon_l)
    print("fiber l :", mjx_data.biomtu.fiber_l)
    print("Muscle Bce:", mjx_data.biomtu.B_ce)
    print("Muscle vm:", mjx_data.biomtu.m)
    print("Fiber acc:", mjx_data.biomtu.fiber_acc)
    print("Fiber v:", mjx_data.biomtu.fiber_v)
    print("Biomtu h:", mjx_data.biomtu.h)
    print(mjx_data.biomtu.v)
    print(mjx_data.biomtu.h)  # The constant high of the muscle.
    print(mjx_data.biomtu.pennation_angle)
    print(mjx_data.biomtu.origin_body_id)
    print(mjx_data.biomtu.insertion_body_id)
    print("mtu act:", mjx_data.biomtu.act)
    # print(mjx_data.biomtu.j)
    print(mjx_data.qfrc_biomtu)
    print(mj_model.key_time)
    print(mj_model.key_qpos)
    print(mj_model.key_qvel)

print_all()

--------------------
BUGMAN: muscle name-glut_med1_r
--------------------
BUGMAN: muscle name-glut_med2_r
--------------------
BUGMAN: muscle name-glut_med3_r
--------------------
BUGMAN: muscle name-bifemlh_r
--------------------
BUGMAN: muscle name-bifemsh_r
--------------------
BUGMAN: muscle name-sar_r
--------------------
BUGMAN: muscle name-add_mag2_r
--------------------
BUGMAN: muscle name-tfl_r
--------------------
BUGMAN: muscle name-pect_r
--------------------
BUGMAN: muscle name-grac_r
--------------------
BUGMAN: muscle name-glut_max1_r
--------------------
BUGMAN: muscle name-glut_max2_r
--------------------
BUGMAN: muscle name-glut_max3_r
--------------------
BUGMAN: muscle name-iliacus_r
--------------------
BUGMAN: muscle name-psoas_r
--------------------
BUGMAN: muscle name-quad_fem_r
--------------------
BUGMAN: muscle name-gem_r
--------------------
BUGMAN: muscle name-peri_r
--------------------
BUGMAN: muscle name-rect_fem_r
--------------------
BUGMAN: muscle nam

In [3]:
print(mjx_model.nbiomtu)
print(mjx_model.nq)
print(mjx_data.qpos)
print(mjx_data.qvel)

54
27
[ 0.      0.95    0.      0.      0.      0.      0.      0.      0.     -0.0036 -0.3957  0.
  0.      0.      0.      0.      0.      0.     -0.0036 -0.3957  0.      0.      0.      0.
  0.      0.      0.    ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Neural Network

In [3]:
import jax
import jax.numpy as jp
import flax
import flax.linen as nn
import optax

class Controller_NN(nn.Module):
    # It will output the mean and variance for each muscle's activation
    nbiomtu = mjx_model.nbiomtu
    def setup(self):
        # Features means the output dimension# Single step

        self.linear1 = nn.Dense(features=400)
        self.linear2 = nn.Dense(features=400)
        self.linear3 = nn.Dense(features=400)
        self.linear4 = nn.Dense(features=400)
        # The last layer will output the mean and logstd
        self.linear5 = nn.Dense(features=self.nbiomtu*2)
        
    
    def __call__(self, x, key):
        x = self.linear1(x)
        x = nn.relu(x)
        x = self.linear2(x)
        x = nn.relu(x)
        x = self.linear3(x)
        x = nn.relu(x)
        x = self.linear4(x)
        x = nn.relu(x)
        x = self.linear5(x)
        # The last layer of the neural requires samping
        mean = x[0:self.nbiomtu]
        logstd = x[self.nbiomtu:self.nbiomtu*2]
        std = jp.exp(logstd)
        samples = jp.clip(jax.random.normal(key)*std*0.3 + mean, 0 ,1)
        
        return samples, mean, logstd


# Test the neural network
control_model = Controller_NN()

print(control_model)
# Init the model
key = jax.random.PRNGKey(66)
sub_keys = jax.random.split(key,1)
# The second parameter is the dommy input
params = control_model.init(key,jp.empty([1,mjx_model.nq*2]),sub_keys[0])
# print(params)
print(control_model.apply( params,jp.ones(mjx_model.nq*2), sub_keys[0]))
jit_nn_apply = jax.jit(lambda params,states,key : control_model.apply(params,states,key))

Controller_NN()
(Array([0.3537, 0.4582, 0.    , 0.    , 0.3568, 0.    , 0.    , 0.    , 0.    , 0.0299, 0.1213,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.0104, 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.4784, 0.    , 0.2003, 0.    , 0.    ,
       0.    , 0.    , 0.2286, 0.    , 0.    , 0.    , 0.    , 0.2364, 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],      dtype=float32), Array([ 0.5507,  0.7117,  0.1663, -0.386 ,  0.572 ,  0.0428,  0.151 , -0.341 , -0.1592,  0.1468,
        0.3125, -0.506 , -0.098 , -0.1812, -0.2472, -0.1027, -0.2657, -0.0098,  0.1604,  0.1802,
       -0.4265, -0.2269, -0.2294, -0.0505,  0.1361,  0.0374,  0.0641, -0.0512,  0.6791, -0.3706,
        0.4092, -0.1719, -0.2406, -0.022 ,  0.1271,  0.4203,  0.0284,  0.0943, -0.4435,  0.1295,
        0.4245,  0.0444, -0.1952,  0.121 , -0.08  , -0.3598, -0.0973, -0.0134,  0.0238, -0.2089,
       -0

## Combine Neural Net and Simulation into one Jax function

In [4]:
# Multiple steps
def step_fn(carry, _):
    data, model = carry
    new_data = mjx.step(model, data)
    new_carry = (new_data, model)
    return new_carry, _

def multiple_steps(model, data):
    init_carry = (data, model)
    y, _ = jax.lax.scan(step_fn, init_carry, None, length=10)
    new_data = y[0]
    return new_data

def nn_mjx_one_step(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel])
    act = jit_nn_apply(nn_params, states, key)[0]
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    mtu = data.biomtu
    mtu = mtu.replace(act = act)
    data = data.replace(biomtu = mtu)
    new_data = mjx.step(model, data)
    return new_data, new_key

@jax.jit
def nn_mjx_multi_steps(nn_params, model, data, key):
    def nn_step_fn(carry, _):
        data, key = carry
        new_data, new_key = nn_mjx_one_step(nn_params, model, data, key)
        new_carry = (new_data, new_key)
        # head_hight = new_data.xpos[2,2]
        # jax.debug.print("Head Height {0}",head_hight)
        return new_carry, _
    init_carry = (data, key)
    y, _ = jax.lax.scan(nn_step_fn, init_carry, None, length=10)
    new_data = y[0]
    new_key = y[1]
    return new_data, new_key

## Testing control model with neural networks

In [6]:
import mujoco.viewer
import time

previous_frame_time = time.time()
i = 0
key = jax.random.key(387)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        # Generate key
        
        mjx_data = mjx_data.replace(xfrc_applied=mj_data.xfrc_applied)
        mjx_data = mjx_data.replace(qpos=mj_data.qpos, qvel=mj_data.qvel, time = mj_data.time)
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': mj_model.opt.gravity,
            'opt.tolerance': mj_model.opt.tolerance,
            'opt.ls_tolerance': mj_model.opt.ls_tolerance,
            'opt.timestep': mj_model.opt.timestep,
        })
        
        # mjx_data = mjx_step(mjx_model, mjx_data)
        mjx_data, key = nn_mjx_multi_steps(params, mjx_model, mjx_data, key)
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        # print("ACT:", mjx_data.biomtu.act)
        # print(mjx_data.qpos)
        print(mjx_data.xpos)
        # print(len(mjx_data.qvel))
        viewer.sync()

Time between frames: 3.808563470840454 seconds
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.0007  0.0005  0.9483]
 [-0.0701 -0.0831  0.8825]
 [-0.0786 -0.0848  0.4871]
 [-0.0714 -0.0874  0.0572]
 [-0.1238 -0.0934  0.0195]
 [ 0.0539 -0.0885  0.0004]
 [-0.0701  0.0839  0.8822]
 [-0.0723  0.0847  0.4865]
 [-0.0764  0.0855  0.0566]
 [-0.1252  0.0907  0.0141]
 [ 0.0534  0.0813  0.0112]
 [-0.0999  0.0007  1.03  ]]
Time between frames: 1.7238128185272217 seconds
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.0039  0.0019  0.943 ]
 [-0.0678 -0.0787  0.8744]
 [-0.0882 -0.0861  0.4799]
 [-0.0768 -0.0979  0.0502]
 [-0.1223 -0.1016  0.0042]
 [ 0.0552 -0.0829  0.0149]
 [-0.0666  0.0882  0.8803]
 [-0.0699  0.0856  0.485 ]
 [-0.0769  0.0822  0.0551]
 [-0.1257  0.0819  0.0125]
 [ 0.0529  0.0733  0.0104]
 [-0.0964 -0.0003  1.025 ]]
Time between frames: 0.02211761474609375 seconds
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.01    0.003   0.9395]
 [-0.0602 -0.076 

## Batched Random Init Model

In [5]:
def random_init(data, model, rng: jax.Array):
    nbiomtu = model.nq
    init_qpos = data.qpos
    init_qvel = data.qvel
    new_rng, rng1, rng2 = jax.random.split(rng, 3)
    # Qpos_1 is the vertical position
    random_qpos = init_qpos + jax.random.uniform(rng1, [nbiomtu], minval=-1, maxval=1)*0.01
    random_qvel = init_qvel + jax.random.uniform(rng2, [nbiomtu], minval=-1, maxval=1)*0.01
    newdata = data.replace(qpos=random_qpos)
    newdata = newdata.replace(qvel=random_qvel)
    newdata = mjx.forward(mjx_model, newdata)
    # print('data:',data.qpos, data.qvel)
    return newdata, new_rng

vrandom_init = jax.jit(jax.vmap(random_init, in_axes=(None, None, 0),out_axes=(0,0)))

## Reward Function

In [6]:
def reward_n_step(nn_params, data, key):
    repeat_n = 150
    def nn_step_fn(carry, _):
        data, key = carry
        new_data, new_key = nn_mjx_one_step(nn_params, mjx_model, data, key)
        new_carry = (new_data, new_key)
        head_hight = new_data.xpos[2,2]
        # jax.debug.print("Head Height {0}",head_hight)
        return new_carry, (head_hight-0.94)**2
    
    def decay_sum_scan(x, decay):
        def f(sxtm1, xt):
            b = xt + decay * sxtm1
            return b, b
        return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]
    
    _, head_heights = jax.lax.scan(nn_step_fn, (data, key), None, repeat_n)
    # new_data, new_key = carry
    reward = decay_sum_scan(head_heights, 0.95)[repeat_n -1]
    return reward
    
reward_grad = jax.jit(jax.grad(reward_n_step))


def batch_reward(nn_params, batched_data, keys):
    return jp.mean(jax.vmap(reward_n_step, (None, 0, 0))(nn_params, batched_data, keys))

batch_reward_grad = jax.jit(jax.grad((batch_reward)))
jit_batch_reward = jax.jit(batch_reward)

In [7]:
# Generate batched data
batch_size = 30
seed = 2024
key = jax.random.key(seed)
rngs = jax.random.split(key, batch_size) 
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)

print("Calculating reward grad")
g = batch_reward_grad(params, mjx_data_batch, rngs)
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)
g = batch_reward_grad(params, mjx_data_batch, rngs)

Calculating reward grad


In [8]:
for i in range(10):
    g = batch_reward_grad(params, mjx_data_batch, rngs)

In [9]:
for i in range(10):
    r = jit_batch_reward(params, mjx_data_batch, rngs)

In [10]:
print(r)
print(g)

0.02389244
{'params': {'linear1': {'bias': Array([-0.0003, -0.0004,  0.0013,  0.0007,  0.0002, -0.0014,  0.    ,  0.0009, -0.0001,  0.0001,
       -0.0003, -0.0005,  0.0006, -0.0039, -0.0014, -0.0001, -0.0003, -0.0001,  0.0004,  0.0003,
        0.0012, -0.0006,  0.0002, -0.0011, -0.0011, -0.0005,  0.0005, -0.0001, -0.001 ,  0.001 ,
        0.0004, -0.    ,  0.001 , -0.0002,  0.0003,  0.0024, -0.0003, -0.0019, -0.0006, -0.0001,
       -0.0023, -0.    , -0.0005,  0.0002, -0.0011, -0.0002, -0.0002,  0.001 , -0.0005,  0.0001,
       -0.0008, -0.0002, -0.0002,  0.0001,  0.0005,  0.0008, -0.    ,  0.0009, -0.    , -0.0017,
       -0.0004, -0.0006, -0.0014, -0.0006, -0.0001,  0.0003, -0.0008, -0.    , -0.    ,  0.0007,
       -0.    ,  0.0012,  0.    , -0.0018, -0.0011, -0.0027,  0.0005,  0.0001,  0.001 ,  0.0001,
       -0.0013, -0.0017,  0.0032,  0.0012, -0.0003,  0.0009, -0.0021, -0.0004,  0.0005,  0.0004,
        0.    ,  0.0003,  0.001 , -0.0003, -0.0003,  0.0002,  0.0001,  0.    ,  0.00

## Train the NN

In [19]:
params = control_model.init(key,jp.empty([1,mjx_model.nq*2]),sub_keys[0])


In [20]:
tx = optax.adam(learning_rate=0.0005)
opt_state = tx.init(params)

for i in range(200):
    # generate random mjx_data
    mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)
    # print(rngs[0])
    r = jit_batch_reward(params, mjx_data_batch, rngs)
    print(r)
    g = batch_reward_grad(params, mjx_data_batch, rngs)
    updates, opt_stats = tx.update(g, opt_state)
    params = optax.apply_updates(params, updates)
    print("params updated")

0.0072576115
params updated
0.18751
params updated
0.0064109424
params updated
0.011019076
params updated
0.040936828
params updated
0.004152858
params updated
0.011928277
params updated
0.00077721546
params updated
0.00045248505
params updated
0.029319387
params updated
0.0043668724
params updated
0.011071529
params updated
0.01370593
params updated
0.0004219494
params updated
0.00791292
params updated
0.0011533484
params updated
0.00028643679
params updated
0.0021521098
params updated
0.00051019597
params updated
0.0017043382
params updated
0.0005594433
params updated
0.00033567532
params updated
0.0006424999
params updated
0.06800821
params updated
0.0006891041
params updated
0.0008310664
params updated
0.0023414264
params updated
0.0011522559
params updated
0.0005724807
params updated
0.00046321852
params updated
0.0007879961
params updated
0.0007597696
params updated
0.00042848833
params updated
0.00048431678
params updated
0.00020580537
params updated
0.00043623213
params updated

KeyboardInterrupt: 

## Test Train

In [13]:
jit_nn_mjx_one_step = jax.jit(nn_mjx_one_step)


In [24]:
import mujoco.viewer
import time


mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
mj_data.qpos = mj_model.key_qpos[0]

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
mjx_data = mjx_step(mjx_model, mjx_data)

previous_frame_time = time.time()
i = 0
key = jax.random.key(seed)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        i += 1
        if(i ==4):
            time.sleep(5)
            # pass
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        # Generate key
        
        mjx_data = mjx_data.replace(xfrc_applied=mj_data.xfrc_applied)
        mjx_data = mjx_data.replace(qpos=mj_data.qpos, qvel=mj_data.qvel, time = mj_data.time)
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': mj_model.opt.gravity,
            'opt.tolerance': mj_model.opt.tolerance,
            'opt.ls_tolerance': mj_model.opt.ls_tolerance,
            'opt.timestep': mj_model.opt.timestep,
        })
        
        # mjx_data = mjx_step(mjx_model, mjx_data)
        mjx_data, key = jit_nn_mjx_one_step(params, mjx_model, mjx_data, key)
        # mjx_data, key = nn_mjx_multi_steps(params, mjx_model, mjx_data, key)
        
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        # print("ACT:", mjx_data.biomtu.act)
        # print(mjx_data.qpos)
        print(mjx_data.xpos[2,2])
        # print(len(mjx_data.qvel))
        viewer.sync()

Time between frames: 0.2678225040435791 seconds
0.9499999
Time between frames: 0.04675865173339844 seconds
0.9500231
Time between frames: 0.011263132095336914 seconds
0.9499774
Time between frames: 5.012398958206177 seconds
0.94981
Time between frames: 0.011791467666625977 seconds
0.94967103
Time between frames: 0.011698007583618164 seconds
0.9495986
Time between frames: 0.009348154067993164 seconds
0.949608
Time between frames: 0.009460687637329102 seconds
0.9496927
Time between frames: 0.011826515197753906 seconds
0.9498546
Time between frames: 0.010198831558227539 seconds
0.9499677
Time between frames: 0.010564327239990234 seconds
0.9496804
Time between frames: 0.008799314498901367 seconds
0.9490548
Time between frames: 0.01104426383972168 seconds
0.94851875
Time between frames: 0.012903451919555664 seconds
0.9478325
Time between frames: 0.010254383087158203 seconds
0.94725096
Time between frames: 0.010094881057739258 seconds
0.94683576
Time between frames: 0.012192964553833008 seco

## Test Gradient for activation

In [14]:
n = mjx_model.nbiomtu
act = jp.ones(n)*0.1

In [28]:
# jax.config.update("jax_debug_nans", True)
for i in range(10):
    g = jit_goal_grad(act, mjx_data, mjx_model)
    # g = goal_grad(act, mjx_data, mjx_model)
print(g)

[ 0.0029  0.0177  0.0011  0.0183  0.0053  0.0064  0.0115  0.02    0.0023  0.0084  0.0302  0.0048
  0.0306  0.0018  0.0038  0.0019  0.0006  0.0057  0.0181  0.0067 -0.0016  0.0128 -0.0306  0.0814
  0.0015  0.0042  0.0169  0.1057  0.0211  0.005   0.0255  0.006   0.021   0.0105  0.0301 -0.0111
  0.0027  0.0128  0.0167  0.0024  0.005   0.0058 -0.0179 -0.1854  0.0519  0.0019  0.0187  0.0343
  0.0252  0.0251 -0.0008  0.0025  0.0002  0.0114]


In [36]:
v_act = jp.ones((50,n))*0.3
for i in range(5):
    v_g = jit_v_goal_grad(v_act,mjx_data,mjx_model)
print(v_g)

2024-10-23 15:14:45.445644: W external/xla/xla/service/hlo_rematerialization.cc:3005] Can't reduce memory use below 6.02GiB (6467407612 bytes) by rematerialization; only reduced to 12.31GiB (13220408549 bytes), down from 12.31GiB (13220408549 bytes) originally
2024-10-23 15:15:46.492778: W external/xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.31GiB (rounded to 13220816896)requested by op 
2024-10-23 15:15:46.493013: W external/xla/xla/tsl/framework/bfc_allocator.cc:508] *___________________________________________________________________________________________________
E1023 15:15:46.493063 2100274 pjrt_stream_executor_client.cc:3084] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 13220816736 bytes.


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 13220816736 bytes.

In [27]:
for i in range(10):
    reward = jit_goal(act, mjx_data, mjx_model)
print(reward)

8.035784


In [ ]:
act = jp.ones(n)*0.0
mtu = mjx_data.biomtu
mtu = mtu.replace(act = act)
test_d = mjx_data
test_d = test_d.replace(biomtu = mtu)
for i in range(100):
    test_d = mjx_step(mjx_model, test_d)
print(test_d.xipos)

In [ ]:
print(test_d.biomtu.f_se)
print(test_d.biomtu.f_ce)
print(test_d.biomtu.l)
print(test_d.biomtu.tendon_l - mjx_model.biomtu_tendon_slack_l)
print(test_d.biomtu.fiber_l - mjx_model.biomtu_ofl)
print(test_d.biomtu.fiber_l)
print(test_d.biomtu.act)

In [64]:
print(mjx_data.xpos)

[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [ 0.     -0.0002  0.95  ]
 [-0.0707 -0.0837  0.8839]
 [-0.0743 -0.0836  0.4882]
 [-0.0743 -0.0835  0.0582]
 [-0.1231 -0.0914  0.0162]
 [ 0.0557 -0.0925  0.0142]
 [-0.0707  0.0833  0.8839]
 [-0.0743  0.0834  0.4882]
 [-0.0743  0.0835  0.0582]
 [-0.1231  0.0914  0.0163]
 [ 0.0557  0.0925  0.0143]
 [-0.1007 -0.0002  1.0315]]


In [ ]:
mjx_data.xpos